
### Задание 1
Возьмите четыре любые темы из корпуса 20newsgroups (постарайтесть брать не слишком похожие, и не слишком разные темы). Векторизуйте датасет с помощью CountVectorizer. Выберете три любых классификатора. Используя кроссвалидацию (любой вариант из KFold, StratifiedKFold, RepeatedStratifiedKFold), подберите оптимальные параметры моделей с помощью grid_search. Обучите классификаторы с оптимальными параметрами. Оцените полученные классификаторы на тесте, мера качества - macro_f1. Посмотрите, насколько полученные результаты на тесте отличаются от результатов предсказания на трейне? - 3 балла

In [103]:
!pip install eli5

In [104]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
import eli5
from eli5.sklearn import PermutationImportance
import warnings
import re
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize
from string import punctuation
warnings.filterwarnings('ignore')


In [0]:
cats = ['sci.electronics', 'sci.med', 'sci.space', 'sci.crypt']

In [0]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)
newsgroups_test = fetch_20newsgroups(subset='test', categories=cats)

In [0]:
count_vect = CountVectorizer()

In [0]:
vectors_train = count_vect.fit_transform(newsgroups_train.data)
vectors_test = count_vect.transform(newsgroups_test.data) 

In [0]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)
stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
repeated_folds = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=20, random_state=0)

###RidgeClassifier

In [111]:
%%time 
rclf = RidgeClassifier()

parameter_grid = {'alpha': [0.001, 0.1, 1, 2, 10]}

grid_search = GridSearchCV(rclf, param_grid=parameter_grid, cv=folds, n_jobs=-1)
grid_search.fit(vectors_train, newsgroups_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
rlcf = RidgeClassifier(**grid_search.best_params_)

Best score: 0.9351256958479593
Best parameters: {'alpha': 10}
CPU times: user 3.92 s, sys: 2.78 s, total: 6.7 s
Wall time: 2min 30s


In [112]:
model1 = rclf.fit(vectors_train, newsgroups_train.target)
train_preds1 = model1.predict(vectors_train)
test_preds1 = model1.predict(vectors_test)
print('result on train: {}'.format(f1_score(newsgroups_train.target, train_preds1, average='macro')))
print('result on test: {}'.format(f1_score(newsgroups_test.target,test_preds1, average='macro')))

result on train: 1.0
result on test: 0.8244203816916094


### LogisticRegression

In [113]:
%%time 
lr = LogisticRegression()

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08, 0.1, 0.15, 1.0],
                  'max_iter': [2,10,20]
                 }

grid_search2 = GridSearchCV(lr, param_grid=parameter_grid, cv=folds, n_jobs=-1)
grid_search2.fit(vectors_train, newsgroups_train.target)
print('Best score: {}'.format(grid_search2.best_score_))
print('Best parameters: {}'.format(grid_search2.best_params_))
lr = LogisticRegression(**grid_search2.best_params_)

Best score: 0.9561872850405985
Best parameters: {'C': 1.0, 'class_weight': None, 'max_iter': 20, 'penalty': 'l2', 'solver': 'liblinear'}
CPU times: user 6.45 s, sys: 1.4 s, total: 7.84 s
Wall time: 11min 35s


In [114]:
model2 = lr.fit(vectors_train, newsgroups_train.target)
train_preds2 = model2.predict(vectors_train)
test_preds2 = model2.predict(vectors_test)
print('result on train: {}'.format(f1_score(newsgroups_train.target, train_preds2, average='macro')))
print('result on test: {}'.format(f1_score(newsgroups_test.target,test_preds2, average='macro')))

result on train: 1.0
result on test: 0.9042118694156432


### SGDClassifier

In [115]:
%%time
sgd = SGDClassifier()

parameter_grid = {'penalty': ['l2', 'l1'], 
                  'alpha': [0.0001, 0.001, 0.01, 0.1], 
                  'max_iter': [1000, 500, 100]}

grid_search3 = GridSearchCV(sgd, param_grid=parameter_grid, cv=folds, n_jobs=-1)
grid_search3.fit(vectors_train, newsgroups_train.target)
print('Best score: {}'.format(grid_search3.best_score_))
print('Best parameters: {}'.format(grid_search3.best_params_))
sgd = SGDClassifier(**grid_search3.best_params_)

Best score: 0.9574477892422791
Best parameters: {'alpha': 0.01, 'max_iter': 100, 'penalty': 'l2'}
CPU times: user 942 ms, sys: 151 ms, total: 1.09 s
Wall time: 4min 9s


In [116]:
model3 = sgd.fit(vectors_train, newsgroups_train.target)
train_preds3 = model3.predict(vectors_train)
test_preds3 = model3.predict(vectors_test)
print('result on train: {}'.format(f1_score(newsgroups_train.target, train_preds3, average='macro')))
print('result on test: {}'.format(f1_score(newsgroups_test.target,test_preds3, average='macro')))

result on train: 1.0
result on test: 0.9032315557065068


### Задание 2
Постройте функцию analyze_features(model, n), которая бы для каждой модели выводила самые значимые признаки по каждому из четырех классов. Для этого вам понадобится словарь, связывающий номер признака с самим признаком, он может быть таким: index_to_word = {v:k for k,v in count_vect.vocabulary_.items()} обращаться к весам модели можно либо через eli5, как мы делали на лекции, либо напрямую: через clf.coef_ - матрица размера (n_classes, n_features), то есть для получения признаков с наибольшим весом для класса n вам нужно сортировать веса внутри n-ной строки. если вы используете деревья решений, то можно воспользоваться методом model.feature_importances - 4 балла

In [0]:
def analyze_features(model, n):
  index_to_word = {v:k for k,v in count_vect.vocabulary_.items()}
  df = eli5.formatters.as_dataframe.explain_weights_df(model)
  
  class1 = df[df['target']==0]
  class2 = df[df['target']==1]
  class3 = df[df['target']==2]
  class4 = df[df['target']==3]
  
  print(n, 'самых значимых признаков класса 1 (crypt):')
  for element in class1.feature[:n]:
    feature = element.strip('x')
    print(index_to_word[int(feature)])
  
  print('\n', n, 'самых значимых признаков класса 2 (electronics):')
  for element in class2.feature[:n]:
    feature = element.strip('x')
    print(index_to_word[int(feature)])

  print('\n', n, 'самых значимых признаков класса 3 (med):')
  for element in class3.feature[:n]:
    feature = element.strip('x')
    print(index_to_word[int(feature)])

  print('\n', n, 'самых значимых признаков класса 4 (space):')
  for element in class4.feature[:n]:
    feature = element.strip('x')
    print(index_to_word[int(feature)])




### Задание 3 
Примените функцию к вашим классификаторам, видны ли по отобранным словам очевидные ошибки? Используйте параметры CountVectorizer, для того, чтобы уменьшить количество признаков и убрать нерелевантные (например числа, токены слишком низкой или высокой документной частотой, и т.д.), постарайтесь добиться улучшения результатов работы моделей (снова выводите результаты модели на трейне и на тесте, чтобы видеть, уменьшается ли переобучение) - 3 балла

In [118]:
analyze_features(model1, 10)

10 самых значимых признаков класса 1 (crypt):
tapped
brinich
gtoal
clipper
dos
code
issa
conference
ebcdic
pollux

 10 самых значимых признаков класса 2 (electronics):
motorola
yxy4145
wayne
exploding
cars
rf
grace
wex
differential
intel

 10 самых значимых признаков класса 3 (med):
photography
roxonal
radford
jmetz
jeeves
doctor
barbecued
amniocentesis
claussen
krillean

 10 самых значимых признаков класса 4 (space):
planets
ryukoku
prb
shread
twist
dgi
jennise
russian
race
divine


In [119]:
analyze_features(model2, 10)

10 самых значимых признаков класса 1 (crypt):
clipper
encryption
key
code
security
gtoal
pgp
chip
dos
nsa

 10 самых значимых признаков класса 2 (electronics):
power
electronics
circuit
radar
tv
motorola
out
line
usa
used

 10 самых значимых признаков класса 3 (med):
doctor
msg
pitt
disease
medical
information
photography
treatment
health
cancer

 10 самых значимых признаков класса 4 (space):
space
orbit
moon
launch
planets
dc
earth
rockets
jennise
dgi


In [120]:
analyze_features(model3, 10)

10 самых значимых признаков класса 1 (crypt):
db
clipper
encryption
anonymous
mov
key
bh
government
security
code

 10 самых значимых признаков класса 2 (electronics):
wire
circuit
wiring
power
ground
used
electronics
tv
neutral
use

 10 самых значимых признаков класса 3 (med):
doctor
disease
health
medical
keyboard
msg
pitt
patients
treatment
cancer

 10 самых значимых признаков класса 4 (space):
space
orbit
moon
earth
launch
solar
satellite
flight
shuttle
telescope


Очевидных ошибок нет. Если выводить топ10 слов, то все они достаточно информативны и хорошо отражают суть своей категории.

### Новый count_vect

In [0]:
noise = stopwords.words('english') + list(punctuation)

In [0]:
count_vect_new = CountVectorizer(max_features=1000, min_df=0.01, max_df=0.4, stop_words=noise)

In [0]:
vectors_train_new = count_vect_new.fit_transform(newsgroups_train.data)
vectors_test_new = count_vect_new.transform(newsgroups_test.data) 

Измененный RidgeClassifier

In [225]:
rclf = RidgeClassifier()

parameter_grid = {'alpha': [0.001, 0.1, 1, 2, 10]}

grid_search = GridSearchCV(rclf, param_grid=parameter_grid, cv=folds, n_jobs=-1)
grid_search.fit(vectors_train_new, newsgroups_train.target)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
rlcf = RidgeClassifier(**grid_search.best_params_)

model1_new = rclf.fit(vectors_train_new, newsgroups_train.target)
train_preds1_new = model1_new.predict(vectors_train_new)
test_preds1_new = model1_new.predict(vectors_test_new)
print('result on train: {}'.format(f1_score(newsgroups_train.target, train_preds1_new, average='macro')))
print('result on test: {}'.format(f1_score(newsgroups_test.target,test_preds1_new, average='macro')))

Best score: 0.8356469169946459
Best parameters: {'alpha': 10}
result on train: 0.9764294635741017
result on test: 0.6782359923989176


Измененная LogisticRegression

In [226]:
%%time 
lr = LogisticRegression()

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08, 0.1, 0.15, 1.0],
                  'max_iter': [2,10,20]
                 }

grid_search2 = GridSearchCV(lr, param_grid=parameter_grid, cv=folds, n_jobs=-1)
grid_search2.fit(vectors_train_new, newsgroups_train.target)
print('Best score: {}'.format(grid_search2.best_score_))
print('Best parameters: {}'.format(grid_search2.best_params_))
lr = LogisticRegression(**grid_search2.best_params_)

model2_new = lr.fit(vectors_train_new, newsgroups_train.target)
train_preds2_new = model2_new.predict(vectors_train_new)
test_preds2_new = model2_new.predict(vectors_test_new)
print('result on train: {}'.format(f1_score(newsgroups_train.target, train_preds2_new, average='macro')))
print('result on test: {}'.format(f1_score(newsgroups_test.target,test_preds2_new, average='macro')))



Best score: 0.9334148849413182
Best parameters: {'C': 1.0, 'class_weight': 'balanced', 'max_iter': 20, 'penalty': 'l2', 'solver': 'liblinear'}
result on train: 1.0
result on test: 0.8633568147882489
CPU times: user 4.35 s, sys: 416 ms, total: 4.76 s
Wall time: 2min 10s


Измененный SGDClassifier

In [227]:
%%time
sgd = SGDClassifier()

parameter_grid = {'penalty': ['l2', 'l1'], 
                  'alpha': [0.0001, 0.001, 0.01, 0.1], 
                  'max_iter': [1000, 500, 100]}

grid_search3 = GridSearchCV(sgd, param_grid=parameter_grid, cv=folds, n_jobs=-1)
grid_search3.fit(vectors_train_new, newsgroups_train.target)
print('Best score: {}'.format(grid_search3.best_score_))
print('Best parameters: {}'.format(grid_search3.best_params_))
sgd = SGDClassifier(**grid_search3.best_params_)

model3_new = sgd.fit(vectors_train_new, newsgroups_train.target)
train_preds3_new = model3_new.predict(vectors_train_new)
test_preds3_new = model3_new.predict(vectors_test_new)
print('result on train: {}'.format(f1_score(newsgroups_train.target, train_preds3_new, average='macro')))
print('result on test: {}'.format(f1_score(newsgroups_test.target,test_preds3_new, average='macro')))



Best score: 0.9342658582420309
Best parameters: {'alpha': 0.01, 'max_iter': 1000, 'penalty': 'l2'}
result on train: 0.9899092922247694
result on test: 0.8618318945019036
CPU times: user 1.16 s, sys: 122 ms, total: 1.29 s
Wall time: 30.2 s


После изменения параметров Countvectorizer переобучение на трейне уменьшилось. Также немного ухудшился результат на тесте. 